# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [361]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from api_keys import g_key
gmaps.configure(api_key=g_key)
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [362]:
weather_data = pd.read_csv("weather data.csv")
weather_data.dropna(inplace=True)
weather_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77.4840,-69.3632,35.38,67,0,5.28,GL,1659856722
1,albany,42.6001,-73.9662,75.72,93,61,2.91,US,1659856453
2,bredasdorp,-34.5322,20.0403,57.15,85,22,12.75,ZA,1659856723
3,kapaa,22.0752,-159.3190,80.92,85,100,3.44,US,1659856724
4,lavrentiya,65.5833,-171.0000,47.26,85,94,8.90,RU,1659856725
5,mataura,-46.1927,168.8643,35.47,97,100,6.76,NZ,1659856726
6,shimoda,34.6667,138.9500,86.36,77,40,7.54,JP,1659856727
7,mar del plata,-38.0023,-57.5575,44.94,77,75,2.30,AR,1659856727
8,alyangula,-13.8483,136.4192,79.74,67,0,15.28,AU,1659856728
9,anadyr,64.7500,177.4833,59.50,82,40,13.42,RU,1659856729


In [363]:
weather_data.count()

City          584
Lat           584
Lng           584
Max Temp      584
Humidity      584
Cloudiness    584
Wind Speed    584
Country       584
Date          584
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [364]:
city_locations = weather_data[["Lat", "Lng"]]

In [365]:
humidity = weather_data["Humidity"]

In [366]:
fig = gmaps.figure()

In [367]:
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, dissipating=False, max_intensity = 10, point_radius = 1)

In [368]:
fig.add_layer(heat_layer)

In [370]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [371]:
hotel_df = weather_data[(weather_data["Max Temp"]<37) & (weather_data["Max Temp"]>10) & (weather_data["Cloudiness"]>70)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [372]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,mataura,-46.1927,168.8643,35.47,97,100,6.76,NZ,1659856726
57,bluff,-46.6000,168.3333,35.80,75,100,21.72,NZ,1659856770
72,tuatapere,-46.1333,167.6833,36.36,89,88,9.08,NZ,1659856721
81,pevek,69.7008,170.3133,32.34,94,100,19.77,RU,1659856789
101,punta arenas,-53.1500,-70.9167,33.91,93,100,8.05,CL,1659856539
409,leningradskiy,69.3833,178.4167,31.96,92,100,17.25,RU,1659857070
471,rio gallegos,-51.6226,-69.2181,32.05,100,100,2.30,AR,1659856895
496,san rafael,-34.6177,-68.3301,36.21,98,100,6.24,AR,1659857139


In [373]:
hotel_df["Hotel Name"] = ""

C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [374]:
#check df to see if column was created
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,mataura,-46.1927,168.8643,35.47,97,100,6.76,NZ,1659856726,
57,bluff,-46.6000,168.3333,35.80,75,100,21.72,NZ,1659856770,
72,tuatapere,-46.1333,167.6833,36.36,89,88,9.08,NZ,1659856721,
81,pevek,69.7008,170.3133,32.34,94,100,19.77,RU,1659856789,
101,punta arenas,-53.1500,-70.9167,33.91,93,100,8.05,CL,1659856539,
409,leningradskiy,69.3833,178.4167,31.96,92,100,17.25,RU,1659857070,
471,rio gallegos,-51.6226,-69.2181,32.05,100,100,2.30,AR,1659856895,
496,san rafael,-34.6177,-68.3301,36.21,98,100,6.24,AR,1659857139,


In [375]:
params = {"key":g_key,
          "radius": 5000
    
}

In [376]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lng"]
    params["location"] = f"{lat}, {lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    try:
        #use index one as 0 contains information on the city, not the hotel name
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
    except (KeyError, IndexError):
        print("None nearby")


C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [377]:
#check to see the type it is stored in
type(hotel_name)

dict

In [378]:
#it is a dictionary, so check to see the keys
hotel_name.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [379]:
hotel_name["results"][1]["name"]

'Hotel San Rafael'

In [380]:
#check to see how many hotels there are
len(hotel_name)

4

In [381]:
#use index 0 to get the first result
hotel_name["results"]

[{'geometry': {'location': {'lat': -34.6128922, 'lng': -68.3390622},
   'viewport': {'northeast': {'lat': -34.59038938226004,
     'lng': -68.2602811831426},
    'southwest': {'lat': -34.66459679665539, 'lng': -68.3901381541867}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'San Rafael',
  'photos': [{'height': 4000,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115878825967078380599">gustavo paredes</a>'],
    'photo_reference': 'AeJbb3fo6Yjtwj4I2X4vkQ5JOHQ0FZRlGTPLXC4KGYV1ZXW4lQvLAZ-a8Jo9e9IqYSQE14zi0zmWqWWvSZKxLuApRaq-VlhathLOeqV9m7_OzHPRzSVoVH1aLd1pzgu0X_tLKFSSB7vspNVDfX_4Gt_ZNQoCBr5JrzZiCRvGAdJaMma_jwuH',
    'width': 3000}],
  'place_id': 'ChIJpwY9sf0HeZYRnTBz1JYn--A',
  'reference': 'ChIJpwY9sf0HeZYRnTBz1JYn--A',
  'scope': 'GOOGLE',
  'types': ['locality', 'political

In [382]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,mataura,-46.1927,168.8643,35.47,97,100,6.76,NZ,1659856726,Ellie's Villa
57,bluff,-46.6000,168.3333,35.80,75,100,21.72,NZ,1659856770,Bluff Homestead - Guesthouse & Campervan Park
72,tuatapere,-46.1333,167.6833,36.36,89,88,9.08,NZ,1659856721,Tuatapere Cemetery
81,pevek,69.7008,170.3133,32.34,94,100,19.77,RU,1659856789,Chaunskaya Rayonnaya Bol'nitsa
101,punta arenas,-53.1500,-70.9167,33.91,93,100,8.05,CL,1659856539,Hotel Hain


In [383]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [384]:
# Add marker layer ontop of heat map
# fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False, max_intensity=10, point_radius = 1)
# fig.add_layer(heat_layer)
# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))